In [1]:
# s3 특정 경로에 있는 파일 이름 리딩
# 정재 후 폴더 생성

In [2]:
import sys
print(sys.path)
print(sys.executable)

['c:\\ProgramData\\anaconda3\\python313.zip', 'c:\\ProgramData\\anaconda3\\DLLs', 'c:\\ProgramData\\anaconda3\\Lib', 'c:\\ProgramData\\anaconda3', '', 'C:\\Users\\USER\\AppData\\Roaming\\Python\\Python313\\site-packages', 'c:\\ProgramData\\anaconda3\\Lib\\site-packages', 'c:\\ProgramData\\anaconda3\\Lib\\site-packages\\win32', 'c:\\ProgramData\\anaconda3\\Lib\\site-packages\\win32\\lib', 'c:\\ProgramData\\anaconda3\\Lib\\site-packages\\Pythonwin']
c:\ProgramData\anaconda3\python.exe


In [3]:
import boto3
import re

# S3 클라이언트 생성
s3 = boto3.client("s3")

bucket_name = "schoolfriends-bym"

# 버킷 내 모든 객체 가져오기
response = s3.list_objects_v2(Bucket=bucket_name)

In [ ]:
for obj in response.get("Contents", []):
    key = obj["Key"]
    if key.endswith("/"):
        print(f"Processing folder: {key}")
    else:
        pass
# 왜 이거밖에 조회가 안됨?

Processing folder: at_basic_inquiry_val/
Processing folder: at_formula_code/
Processing folder: at_low_scale/
Processing folder: at_question/
Processing folder: at_question_choice/
Processing folder: at_question_extra/
Processing folder: at_scale_code/
Processing folder: at_sub_psy_item/
Processing folder: at_user_formula_score/
Processing folder: at_user_scale_score/
Processing folder: at_user_testing_paper/
Processing folder: at_user_testing_paper_pn/
Processing folder: at_user_testing_paper_sct/
Processing folder: at_user_testing_paper_tr/
Processing folder: member/
Processing folder: school_class/


In [5]:
for obj in response.get("Contents", []):
    print(obj["Key"])

at_basic_inquiry_val/
at_basic_inquiry_val/AT_BASIC_INQUIRY_VAL_202508291453.csv
at_formula_code/
at_formula_code/AT_FORMULA_CODE_202508291455.csv
at_low_scale/
at_low_scale/AT_LOW_SCALE_202508291455.csv
at_question/
at_question/AT_QUESTION_202508291458.csv
at_question_choice/
at_question_choice/AT_QUESTION_CHOICE_202508291459.csv
at_question_extra/
at_question_extra/AT_QUESTION_EXTRA_202508291500.csv
at_scale_code/
at_scale_code/AT_SCALE_CODE_202508291500.csv
at_sub_psy_item/
at_sub_psy_item/AT_SUB_PSY_ITEM_202508291500.csv
at_user_formula_score/
at_user_formula_score/AT_USER_FORMULA_SCORE_202508291501.csv
at_user_scale_score/
at_user_scale_score/AT_USER_SCALE_SCORE_202508291503.csv
at_user_testing_paper/
at_user_testing_paper/AT_USER_TESTING_PAPER_202508291503.csv
at_user_testing_paper_pn/
at_user_testing_paper_pn/AT_USER_TESTING_PAPER_PN_202508291504.csv
at_user_testing_paper_sct/
at_user_testing_paper_sct/AT_USER_TESTING_PAPER_SCT_202508291505.csv
at_user_testing_paper_tr/
at_user_

In [ ]:
# CSV 파일만 폴더 만들어서 이동 및 삭제 처리
for obj in response.get("Contents", []):
    key = obj["Key"]
    if "/" not in key and key.lower().endswith(".csv"):
        print(f"Processing file: {key}")
        match = re.search(r"([A-Z]+)_([A-Z]+)", key)
        print(match)
        folder_name = match.group(0).lower()
        print(folder_name)
        new_key = f"{folder_name}/{key}"
        print(f"Moving {key} to {new_key}")

        # S3 내에서 파일 복사
        s3.copy_object(
            Bucket=bucket_name,
            CopySource={"Bucket": bucket_name, "Key": key},
            Key=new_key
        )

        # 원본 삭제
        s3.delete_object(Bucket=bucket_name, Key=key)
    else:
        pass

Processing file: PSY_CLASS_202508291509.csv
<re.Match object; span=(0, 9), match='PSY_CLASS'>
psy_class
Moving PSY_CLASS_202508291509.csv to psy_class/PSY_CLASS_202508291509.csv
Processing file: PSY_ESTIMATE_202508291511.csv
<re.Match object; span=(0, 12), match='PSY_ESTIMATE'>
psy_estimate
Moving PSY_ESTIMATE_202508291511.csv to psy_estimate/PSY_ESTIMATE_202508291511.csv
Processing file: PSY_ORD_202508291511.csv
<re.Match object; span=(0, 7), match='PSY_ORD'>
psy_ord
Moving PSY_ORD_202508291511.csv to psy_ord/PSY_ORD_202508291511.csv
Processing file: PSY_ORD_CLASS_202508291511.csv
<re.Match object; span=(0, 7), match='PSY_ORD'>
psy_ord
Moving PSY_ORD_CLASS_202508291511.csv to psy_ord/PSY_ORD_CLASS_202508291511.csv
Processing file: PSY_TARGET_DETAIL_202508291511.csv
<re.Match object; span=(0, 10), match='PSY_TARGET'>
psy_target
Moving PSY_TARGET_DETAIL_202508291511.csv to psy_target/PSY_TARGET_DETAIL_202508291511.csv
Processing file: SCHOOL_INFO_202508291512.csv
<re.Match object; span=

In [30]:
# 폴더 이름 리스트화
s3 = boto3.client("s3")
bucket_name = "schoolfriends-bym"
response = s3.list_objects_v2(Bucket=bucket_name)
folder_names = []
for obj in response.get("Contents", []):
    key = obj["Key"]
    if "/" in key and key.endswith("/"):
        table_name = key.split("/")[0]
        folder_names.append(table_name)
    else:
        pass
print('s3에 있는 폴더명 :', folder_names)

# s3 폴더 별로 크롤러 만들어서 아테네 테이블 만들때
glue = boto3.client("glue", region_name="ap-northeast-2")

# 공통 파라미터
role_name = 'arn:aws:iam::932744610695:role/service-role/AWSGlueServiceRole-Schoolfriends'
database_name = 'schoolfriends'
bucket_name = 'schoolfriends-bym'

for folder in folder_names:
    crawler_name = f"crawler_{folder}"
    s3_target_path = f"s3://{bucket_name}/{folder}/"
    
    try:
        response = glue.create_crawler(
            Name=crawler_name,
            Role=role_name,
            DatabaseName=database_name,
            Targets={'S3Targets': [{'Path': s3_target_path}]},
            SchemaChangePolicy={
                'UpdateBehavior': 'UPDATE_IN_DATABASE',
                'DeleteBehavior': 'DEPRECATE_IN_DATABASE'
            }
        )
        print(f"Crawler '{crawler_name}' created successfully.")
    except glue.exceptions.AlreadyExistsException:
        print(f"Crawler '{crawler_name}' already exists.")
    except Exception as e:
        print(f"Error creating crawler '{crawler_name}': {e}")

s3에 있는 폴더명 : ['at_basic_inquiry_val', 'at_formula_code', 'at_low_scale', 'at_question', 'at_question_choice', 'at_question_extra', 'at_scale_code', 'at_sub_psy_item', 'at_user_formula_score', 'at_user_scale_score', 'at_user_testing_paper', 'at_user_testing_paper_pn', 'at_user_testing_paper_sct', 'at_user_testing_paper_tr', 'member', 'school_class']
Crawler 'crawler_at_basic_inquiry_val' created successfully.
Crawler 'crawler_at_formula_code' created successfully.
Crawler 'crawler_at_low_scale' created successfully.
Crawler 'crawler_at_question' created successfully.
Crawler 'crawler_at_question_choice' created successfully.
Crawler 'crawler_at_question_extra' created successfully.
Crawler 'crawler_at_scale_code' created successfully.
Crawler 'crawler_at_sub_psy_item' created successfully.
Crawler 'crawler_at_user_formula_score' created successfully.
Crawler 'crawler_at_user_scale_score' created successfully.
Crawler 'crawler_at_user_testing_paper' created successfully.
Crawler 'crawler

In [33]:
# 각 크롤러의 속성을 변경하고 싶은 경우
# - 이미 생성된 Glue의 크롤러의 속성을 변경하고 싶은 경우
# - 예를 들어 스키마 구조 변경 정책(SchemaChangePolicy) 등 일괄 변경
import boto3

glue = boto3.client("glue", region_name="ap-northeast-2")

# 크롤러 이름 리스트 가져오기
response = glue.list_crawlers()
crawler_names = response.get("CrawlerNames", [])
print(crawler_names)

['amazon_reviews_partitioned_crawler', 'crawler_at_basic_inquiry_val', 'crawler_at_formula_code', 'crawler_at_low_scale', 'crawler_at_question', 'crawler_at_question_choice', 'crawler_at_question_extra', 'crawler_at_scale_code', 'crawler_at_sub_psy_item', 'crawler_at_user_formula_score', 'crawler_at_user_scale_score', 'crawler_at_user_testing_paper', 'crawler_at_user_testing_paper_pn', 'crawler_at_user_testing_paper_sct', 'crawler_at_user_testing_paper_tr', 'crawler_member', 'crawler_school_class', 'schoolfriends_member_crawler']


In [35]:
import time
for crawler in crawler_names:
    if crawler.startswith("crawler_"):
        try:
            glue.update_crawler(
                Name=crawler,
                SchemaChangePolicy={
                    "UpdateBehavior": "LOG",
                    "DeleteBehavior": "LOG"
                }
            )
            print(f"✅ {crawler} 의 SchemaChangePolicy 업데이트 완료")
            time.sleep(1)
        except Exception as e:
            print(f"❌ {crawler} 업데이트 실패: {repr(e)}")

✅ crawler_at_basic_inquiry_val 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_formula_code 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_low_scale 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_question 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_question_choice 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_question_extra 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_scale_code 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_sub_psy_item 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_user_formula_score 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_user_scale_score 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_user_testing_paper 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_user_testing_paper_pn 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_user_testing_paper_sct 의 SchemaChangePolicy 업데이트 완료
✅ crawler_at_user_testing_paper_tr 의 SchemaChangePolicy 업데이트 완료
✅ crawler_member 의 SchemaChangePolicy 업데이트 완료
✅ crawler_school_class 의 SchemaChangePolicy 업데이트 완료
